In [1]:
from pathlib import Path
import csv
from collections import defaultdict
import os.path
from statistics import mean 
import pandas as pd
import altair as alt
import fiona
import geopandas as gpd
import gpdvega
import json

from matplotlib import cm
from matplotlib.colors import to_hex

colormap = cm.spring

In [2]:
alt.renderers.enable('notebook')
alt.themes.enable('opaque')
alt.data_transformers.enable('gpd_to_values')

DataTransformerRegistry.enable('gpd_to_values')

In [3]:
def read_csv(filename):
    with open(filename, newline='') as File:  
        reader = csv.reader(File)
        header1 = next(reader)
        header2 = next(reader)
        results = []
        for row in reader:
            folder = Path(row[0].strip("'"))
            parts = folder.parts
            if len(parts) > 2 and parts[-1].lower().endswith('.tif'):
                results.append({"folder": str(folder.parent), "hits": int(row[1]), "bytes": int(row[2]), "cost": float(row[3])})
        return results

In [4]:
all_entries = read_csv("Files_20190417_20190423.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'Files_20190417_20190423.csv'

In [5]:
all_entries = read_csv("Files_20190407_20190506.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'Files_20190407_20190506.csv'

In [ ]:
len(all_entries)

In [ ]:
def product_name(folder):
    parts = Path(folder).parts
    if parts[0] == 'mangrove_cover':
        return parts[0]
    return os.path.join(*parts[:2])


def spatial_id(folder):
    parts = Path(folder).parts
    if len(parts) > 2 and parts[0] == 'L2' and parts[1] == 'sentinel-2-nrt' and parts[-1] in ['NBAR', 'NBART', 'QA', 'SUPPLEMENTARY', 'LAMBERTIAN']:
        try:
            return parts[-2].split("_")[-2][1:]
        except IndexError:
            print(folder)
    else:
        return '<none>'

def merge_pre(folder, dicts):
    return {
        'hits': max(int(d['hits']) for d in dicts),
        'bytes': sum(int(d['bytes']) for d in dicts),
        'cost': sum(float(d['cost']) for d in dicts),
        'spatial_id': spatial_id(folder),
        'product': product_name(folder),
        'folder': folder
    }

def group(entry_list, key):
    lookup = defaultdict(list)
    
    for d in entry_list:
        lookup[d[key]].append(d)
        
    return lookup
    

def merge(dicts):
    return {
        'hits': sum(int(d['hits']) for d in dicts),
        'bytes': sum(int(d['bytes']) for d in dicts),
        'cost': sum(float(d['cost']) for d in dicts)
    }

In [ ]:
stage1 = [merge_pre(key, value) for key, value in group(all_entries, 'folder').items()]

In [ ]:
sentinel2 = [d for d in stage1 if d['product'] == 'L2/sentinel-2-nrt']

In [ ]:
s2_summary = {key: merge(value)['hits'] for key, value in group(sentinel2, 'spatial_id').items()}

In [ ]:
s2_summary['56HLH']

In [ ]:
s2_summary_list = list(s2_summary.items())

In [ ]:
s2_mgrs = [entry[0] for entry in s2_summary_list]
s2_hits = [entry[1] for entry in s2_summary_list]

In [ ]:
hit_values = pd.DataFrame({'MGRS': s2_mgrs, 'hits': s2_hits})

In [ ]:
grid = fiona.open('australia-mgrs-grid.geojson')

In [ ]:
with open('australia-mgrs-grid.geojson') as fl:
    input_gj = json.load(fl)

In [ ]:
feats = input_gj['features']

In [ ]:
for feat in feats:
    if 'MGRS' in feat['properties']:
        mgrs = feat['properties']['MGRS']
        if mgrs in s2_summary:
            feat['properties']['hits'] = s2_summary[mgrs]
            feat['properties']['stroke'] = "#ff00ff"
            feat['properties']['fill-opacity'] = 0.5
            feat['properties']['fill'] = "#ff0000"
            if feat['properties']['hits'] > 1000:
                feat['properties']['fill'] = "#0000ff"
                

In [ ]:
with open('sentinel-2-hitmap-test.geojson', 'w') as fl:
    json.dump(input_gj, fl)

In [ ]:
ex_feat = feats[0]

In [ ]:
list(ex_feat['properties'])

In [ ]:
with open('low_composite_20.geojson') as fl:
    low_comp = json.load(fl)

In [ ]:
list(low_comp['features'][0]['properties'])

In [ ]:
mgrs_poly = {}

In [ ]:
for entry in grid:
    mgrs = entry['properties']['MGRS']
    mgrs_poly[mgrs] = entry

In [ ]:
c234 = set(mgrs_poly)

In [ ]:
geojson2 = [mgrs_poly[x] for x in c234]

In [ ]:
common = set(s2_summary) & set(mgrs_poly)

In [ ]:
geojson = [mgrs_poly[x] for x in common]

In [ ]:
alt.Chart(alt.Data(values=geojson)).mark_geoshape(stroke='black', strokeWidth=1).encode().properties(width=400, height=400)

In [ ]:
polygons = gpd.GeoDataFrame.from_features((geojson))

In [ ]:
polygons.head()

In [ ]:
alt.Chart(merged).mark_geoshape(
        stroke='black',
        strokeWidth=1
    ).encode(
    ).properties(
        width=400,
        height=400
    )

In [ ]:
merged = polygons.merge(hit_values, on= 'MGRS', how = 'inner')[['geometry', 'hits', 'MGRS']]

In [ ]:
merged.head()

In [ ]:
def gen_map(geodata, color_column, title):
    '''Generates DC ANC map with population choropleth and ANC labels'''
    # Add Base Layer
    base = alt.Chart(geodata, title = title).mark_geoshape(
        stroke='black',
        strokeWidth=1
    ).encode(
    ).properties(
        width=400,
        height=400
    )
    # Add Choropleth Layer
    choro = alt.Chart(geodata).mark_geoshape(
        fill='lightgray',
        stroke='black'
    ).encode(
        alt.Color(color_column, 
                  type='quantitative', 
                 scale=alt.Scale(scheme='bluegreen'),
                 title = "hits")
    )

    return base + choro


In [ ]:
gen_map(merged, 'hits', 'sentinel-2')

In [ ]:
gen_map(polygons, 'hits', 'sentinel-2')

In [ ]:

  
def gen_base(geojson):
    '''Generates baselayer of DC ANC map'''
    base = alt.Chart(alt.Data(values=geojson)).mark_geoshape(
        stroke='black',
        strokeWidth=1
    ).encode(
    ).properties(
        width=400,
        height=400
    )
    return base



In [ ]:
import requests

In [ ]:
geos = "australia-mgrs-grid.geojson"

In [ ]:
gen_base(grid)

In [ ]:
polygons.plot()

In [ ]:
geojson

In [ ]:
gdf_js = gpd.read_file("australia-mgrs-grid.geojson")

In [ ]:
gdf_js

In [ ]:
gdf_js.shape

In [ ]:
gdf_js.plot()

In [ ]:
%matplotlib

In [ ]:
gdf_js.GZD.unique()

In [ ]:
from matplotlib import cm
from matplotlib.colors import to_hex

In [ ]:
to_hex(colormap(1))

In [ ]:
to_hex(colormap(0))